In [7]:
import numpy as np
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import *
from tensorflow.keras import utils
from keras.layers import CuDNNLSTM
from tensorflow.keras.models import Sequential, Model
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


In [5]:
#file = "D:\\tuc\\exam10\\Thesis\\Dataset\\CMU_clips_no_zip.npy"
file = '/home/kyriakos/Desktop/Projects/Dataset/merged.npy'
X = np.load(file, allow_pickle=True)
X = np.reshape(X, (X.shape[0], X.shape[1],X.shape[2]*X.shape[3]))
X = X/np.max(X)
X1=X.shape[0]
X2=X.shape[1]
X3=X.shape[2]
#X=np.abs(X)

1073 3198 186


In [3]:

#file = "D:\\tuc\\exam10\\Thesis\\Dataset\\y.npy"
file = '/home/kyriakos/Desktop/Projects/Dataset/y_class.npy'
Y = np.load(file, allow_pickle=True)
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(Y)
print(Y)


[ 0 53  0 ...  0  0 36]


In [4]:
file = '/home/kyriakos/Desktop/Projects/Dataset/y.npy'
Y = np.load(file, allow_pickle=True)
print(Y)

[0. 1. 0. ... 0. 0. 1.]


In [4]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size = 0.2, random_state = 42)

In [5]:
X_train = X_train.reshape(858, 3198, 186,1)
y_train = y_train.reshape(858,1)
X_test = X_test.reshape(215, 3198, 186,1)
y_test = y_test.reshape(215,1)

In [5]:
print(X_train.shape[1:])

(3198, 186, 1)


In [5]:
del X


In [11]:

noise_shape = (100,)
model = Sequential()
model.add(Dense(X2 * X3, activation="relu", input_shape=noise_shape))
model.add(Reshape((X2, X3)))
model.add(BatchNormalization(momentum=0.8))
model.add(UpSampling3D())
model.add(Convolution3D(128, kernel_size=3, padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(momentum=0.8)) 
model.add(UpSampling3D())
model.add(Convolution3D(64, kernel_size=3, padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(momentum=0.8))
model.add(Convolution3D(1, kernel_size=3, padding="same"))
model.add(Activation("tanh"))	
model.summary()
noise = Input(shape = noise_shape)
image = model(noise)
discriminator = Model(noise, image)

IndexError: Exception encountered when calling layer "up_sampling3d" (type UpSampling3D).

list index out of range

Call arguments received:
  • inputs=tf.Tensor(shape=(None, 3198, 186), dtype=float32)

In [8]:


nNeurons = 186
output_layers = 68
#output_layers = 1
# creating the NN model
model = Sequential()
model.add( LSTM( nNeurons,activation = 'tanh' ,return_sequences=True,  input_shape = (3198, 186) ) )
model.add( Dropout(0.2))
model.add( LSTM( nNeurons ,activation = 'tanh') )
model.add( Dropout(0.2))
model.add(Flatten())
model.add( Dense( output_layers, activation = 'softmax' ) )

opt = tf.keras.optimizers.Adam(learning_rate =1e-3, decay=1e-5)
model.compile( loss = 'mse',  optimizer = opt, metrics = ['accuracy'] )
model.summary()

#probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
#filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-biggeer.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose = 1, save_best_only=True, mode = 'min')
#callbacks_list = [checkpoint]

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 3198, 186)         277512    
                                                                 
 dropout_2 (Dropout)         (None, 3198, 186)         0         
                                                                 
 lstm_3 (LSTM)               (None, 186)               277512    
                                                                 
 dropout_3 (Dropout)         (None, 186)               0         
                                                                 
 flatten_1 (Flatten)         (None, 186)               0         
                                                                 
 dense_1 (Dense)             (None, 68)                12716     
                                                                 
Total params: 567,740
Trainable params: 567,740
Non-tr

In [9]:

history = model.fit(X_train, y_train,validation_data = (X_test,y_test), epochs=15, batch_size=1)
#h=model.fit(X_train, y_train, epochs = 30, batch_size=64) #callbacks=callbacks_list)
#model.save('cmuclassifier.model')
y_pred = model.predict(X_test)

#y_pred = model.predict(X_test)

2021-12-31 17:57:01.719114: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2041449696 exceeds 10% of free system memory.


Epoch 1/15
858/858 [==============================] - ETA: 0s - loss: 829.3969 - accuracy: 0.0117

2021-12-31 18:34:04.257420: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 511552080 exceeds 10% of free system memory.


858/858 [==============================] - 2297s 3s/step - loss: 829.3969 - accuracy: 0.0117 - val_loss: 885.2687 - val_accuracy: 0.0140
Epoch 2/15
858/858 [==============================] - 2265s 3s/step - loss: 829.3972 - accuracy: 0.0152 - val_loss: 885.2687 - val_accuracy: 0.0326
Epoch 3/15
858/858 [==============================] - 2302s 3s/step - loss: 829.3965 - accuracy: 0.0117 - val_loss: 885.2687 - val_accuracy: 0.0140
Epoch 4/15
858/858 [==============================] - 2299s 3s/step - loss: 829.3969 - accuracy: 0.0128 - val_loss: 885.2687 - val_accuracy: 0.0047
Epoch 5/15
858/858 [==============================] - 2300s 3s/step - loss: 829.3970 - accuracy: 0.0117 - val_loss: 885.2687 - val_accuracy: 0.0093
Epoch 6/15
858/858 [==============================] - 2277s 3s/step - loss: 829.3970 - accuracy: 0.0093 - val_loss: 885.2687 - val_accuracy: 0.0326
Epoch 7/15
858/858 [==============================] - 2293s 3s/step - loss: 829.3968 - accuracy: 0.0186 - val_loss: 885.268

KeyboardInterrupt: 

In [8]:
pred = []
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
#Converting one hot encoded test label to label
test = []
for i in range(len(y_test)):
    test.append(np.argmax(y_test[i]))


a = accuracy_score(pred,test)
print('Accuracy is:', a*100)

2021-12-31 14:39:45.470701: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 511552080 exceeds 10% of free system memory.


Accuracy is: 0.9302325581395349


In [11]:
#Converting predictions to label

history = model.fit(X_train, y_train,validation_data = (X_test,y_test), epochs=3, batch_size=128)



plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(history.history['loss']) 
plt.plot(history.history['val_loss']) 
plt.title('Model loss') 
plt.ylabel('Loss') 
plt.xlabel('Epoch') 
plt.legend(['Train', 'Test'], loc='upper left') 
plt.show()


2021-12-28 11:49:16.435990: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2041449696 exceeds 10% of free system memory.


Epoch 1/3
7/7 [==============================] - 25s 4s/step - loss: 0.4359 - accuracy: 0.5641 - val_loss: 0.3953 - val_accuracy: 0.6047
Epoch 2/3
7/7 [==============================] - 25s 4s/step - loss: 0.4359 - accuracy: 0.5641 - val_loss: 0.3953 - val_accuracy: 0.6047
Epoch 3/3
7/7 [==============================] - 29s 4s/step - loss: 0.4359 - accuracy: 0.5641 - val_loss: 0.3953 - val_accuracy: 0.6047


KeyError: 'acc'

In [2]:
mnist = tf.keras.datasets.mnist  # mnist is a dataset of 28x28 images of handwritten digits and their labels
(x_train, y_train),(x_test, y_test) = mnist.load_data()  # unpacks images to x_train/x_test and labels to y_train/y_test

x_train = x_train/255.0
x_test = x_test/255.0

print(x_train.shape)
print(x_train[0].shape)

(60000, 28, 28)
(28, 28)


In [3]:
model = Sequential()
model.add(LSTM(16, input_shape=(x_train.shape[1:]), activation='tanh', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(16, activation='tanh'))
model.add(Dropout(0.1))

model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 28, 16)            2880      
                                                                 
 dropout (Dropout)           (None, 28, 16)            0         
                                                                 
 lstm_1 (LSTM)               (None, 16)                2112      
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                1

In [4]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

model.fit(x_train,
          y_train,
          epochs=3,
          validation_data=(x_test, y_test))

Epoch 1/3
1875/1875 [==============================] - 23s 10ms/step - loss: 1.1201 - accuracy: 0.6109 - val_loss: 0.4455 - val_accuracy: 0.8755
Epoch 2/3
1875/1875 [==============================] - 17s 9ms/step - loss: 0.5410 - accuracy: 0.8378 - val_loss: 0.2522 - val_accuracy: 0.9310
Epoch 3/3
1875/1875 [==============================] - 18s 9ms/step - loss: 0.4005 - accuracy: 0.8856 - val_loss: 0.1965 - val_accuracy: 0.9430
